In [1]:
import torch
from torchtext.vocab import Vectors

/home/prajwal/NLP and GenAI/env/lib/python3.10/site-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/home/prajwal/NLP and GenAI/env/lib/python3.10/site-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)


In [2]:
class newGloVe(Vectors):
    url = {
        "42B": "https://huggingface.co/stanfordnlp/glove/resolve/main/glove.42B.300d.zip",
        "840B": "https://huggingface.co/stanfordnlp/glove/resolve/main/glove.840B.300d.zip",
        "twitter.27B": "https://huggingface.co/stanfordnlp/glove/resolve/main/glove.twitter.27B.zip",
        "6B": "https://huggingface.co/stanfordnlp/glove/resolve/main/glove.6B.zip",
    }

    def __init__(self, name="840B", dim=300, **kwargs) -> None:
        url = self.url[name]
        print(f"Downloading from {url}")
        name = "glove.{}.{}d.txt".format(name, str(dim))
        super(newGloVe, self).__init__(name, url=url, **kwargs)


glove = newGloVe(name="6B", dim=100)

.vector_cache/glove.6B.zip: 862MB [01:23, 10.3MB/s]                               
100%|█████████▉| 400000/400001 [00:10<00:00, 38365.68it/s]


In [3]:
# number of words and embeddings
glove.vectors.shape

torch.Size([400001, 100])

In [4]:
# get an embedding vector
def get_embedding_vector(word):
    word_index = glove.stoi[word]
    emb = glove.vectors[word_index]
    return emb

In [5]:
get_embedding_vector("chess").shape

torch.Size([100])

In [6]:
# find closest words from input word
def get_closest_words_from_word(word, max_n=5):
    word_emb = get_embedding_vector(word)
    distances = [
        (w, torch.dist(word_emb, get_embedding_vector(w)).cpu().item())
        for w in glove.itos
    ]
    dist_sort_filt = sorted(distances, key=lambda x: x[1])[:max_n]
    return dist_sort_filt

In [7]:
get_closest_words_from_word("chess")

[('chess', 0.0),
 ('backgammon', 4.379469394683838),
 ('grandmasters', 4.56368350982666),
 ('grandmaster', 4.613785743713379),
 ('scrabble', 4.677640438079834)]

In [11]:
get_closest_words_from_word("bangalore")

[('bangalore', 0.0),
 ('chennai', 2.703354835510254),
 ('hyderabad', 3.191100597381592),
 ('kolkata', 3.500156879425049),
 ('ahmedabad', 3.6202988624572754)]

In [8]:
# find closest words from embedding
def get_closest_words_from_embedding(word_emb, max_n=5):
    distances = [
        (w, torch.dist(word_emb, get_embedding_vector(w)).cpu().item())
        for w in glove.itos
    ]
    dist_sort_filt = sorted(distances, key=lambda x: x[1])[:max_n]
    return dist_sort_filt

In [9]:
# find word analogies
# e.g. King is to Queen like Man is to Woman
def get_word_analogy(word1, word2, word3, max_n=5):
    # logic w1= king, ...
    # w1 - w2 + w3 --> w4
    word1_emb = get_embedding_vector(word1)
    word2_emb = get_embedding_vector(word2)
    word3_emb = get_embedding_vector(word3)
    word4_emb = word1_emb - word2_emb + word3_emb
    analogy = get_closest_words_from_embedding(word4_emb)
    return analogy

In [10]:
get_word_analogy(word1="sister", word2="brother", word3="nephew")

[('sister', 2.5398471355438232),
 ('niece', 2.700707197189331),
 ('granddaughter', 3.272900342941284),
 ('sister-in-law', 3.652935743331909),
 ('cousin', 3.657327651977539)]